[View in Colaboratory](https://colab.research.google.com/github/nishanthjois/Deep-Learning-Advanced/blob/master/MNIST_Tensorflow_Estimator.ipynb)

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

# Our application logic will be added here

# if __name__ == "__main__":
#   tf.app.run()

Convolutional Layer #1: Applies 32 5x5 filters (extracting 5x5-pixel subregions), with ReLU activation function

Pooling Layer #1: Performs max pooling with a 2x2 filter and stride of 2 (which specifies that pooled regions do not overlap)

Convolutional Layer #2: Applies 64 5x5 filters, with ReLU activation function

Pooling Layer #2: Again, performs max pooling with a 2x2 filter and stride of 2

Dense Layer #1: 1,024 neurons, with dropout regularization rate of 0.4 (probability of 0.4 that any given element will be dropped during training)

Dense Layer #2 (Logits Layer): 10 neurons, one for each digit target class (0–9).

In [0]:
def cnn_model(features,labels,mode):
  
  
  input_layer = tf.reshape(features["x"],[-1,28,28,1])
  
  conv1 = tf.layers.conv2d(inputs=input_layer,filters=32,kernel_size=[5,5],padding="same",activation=tf.nn.relu)
  
  pool1 = tf.layers.max_pooling2d(inputs=conv1,pool_size=[2,2],strides=2)
  
  conv2=tf.layers.conv2d(
  
         inputs=pool1,
         filters = 64,
         padding="same",
         activation=tf.nn.relu,
         kernel_size=[5,5]
  )
  
  pool2=tf.layers.max_pooling2d(inputs=conv2,pool_size=[2,2],strides=2)
  
  pool2_flat=tf.reshape(pool2,[-1,7*7*64])
  
  dense=tf.layers.dense(inputs=pool2_flat,units=1024,activation=tf.nn.relu)
  
  dropout = tf.layers.dropout(inputs=dense,rate=0.4,training=mode==tf.estimator.ModeKeys.TRAIN)
  
  logits = tf.layers.dense(inputs=dropout,units=10)
  
  predictions={
      'classes': tf.argmax(input=logits,axis=1),
      'probabilities':tf.nn.softmax(logits,name='softmax_tensor')
      
  }
  
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode,predictions=predictions)
  
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels,logits=logits)
  
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer=tf.train.GradientDescentOptimizer(learning_rate=.001)
    train_op=optimizer.minimize(loss=loss,
                               global_step=tf.train.get_global_step())
    
    return tf.estimator.EstimatorSpec(mode=mode,loss=loss,train_op=train_op)
  
  # Eval
  eval_metric_ops={
      'accuracy':tf.metrics.accuracy(labels=labels,predictions=predictions["classes"])
      
  }
  
  return tf.estimator.EstimatorSpec(mode=mode,loss=loss,eval_metric_ops=eval_metric_ops)

In [20]:

mnist = tf.contrib.learn.datasets.load_dataset('mnist')
train_data=mnist.train.images
train_labels=np.asarray(mnist.train.labels,dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)


Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST-data/t10k-labels-idx1-u

In [38]:
# Create Estimator
mnist_classifier=tf.estimator.Estimator(model_fn=cnn_model
                                       ,model_dir="/tmp/mnist_convnet_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe56bd25080>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
tensors_to_log={'probabilities':'softmax_tensor'}
logging_hook = tf.train.LoggingTensorHook(
                tensors=tensors_to_log,every_n_iter=50)

In [0]:
train_input_fn=tf.estimator.inputs.numpy_input_fn(

                x={'x':train_data},
                y=train_labels,
                batch_size=100,
                num_epochs=2,
                shuffle=True

)

In [43]:
mnist_classifier.train(input_fn=train_input_fn,steps=20000,hooks=[logging_hook])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-1100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1101 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:probabilities = [[0.0697405  0.09473851 0.0883764  0.11932074 0.09984111 0.08140025
  0.06921818 0.12783316 0.12029365 0.12923753]
 [0.12707517 0.08047608 0.10561758 0.12240012 0.09263569 0.07695897
  0.06208078 0.1172921  0.11331275 0.10215076]
 [0.07135919 0.08393985 0.08364664 0.08194012 0.1244434  0.07451244
  0.09506114 0.13448194 0.12148257 0.12913272]
 [0.05191788 0.09753399 0.11149695 0.10550299 0.11237521 0.07584494
  0.08168646 0.13251331 0.11314669 0.11798158]
 [0.09794028 0.13245595 0.15296358 0.09124402 0.08244672 0.07845871
  0.08418819 0.06565352 0.11581661

INFO:tensorflow:probabilities = [[0.06792632 0.0828743  0.17120628 0.13690312 0.08609096 0.05459307
  0.11898185 0.1000675  0.08516004 0.09619658]
 [0.08902135 0.07982104 0.09085283 0.08198451 0.14615163 0.0713535
  0.12245945 0.12241593 0.11503335 0.08090644]
 [0.17767732 0.05463477 0.09043766 0.10051002 0.12618671 0.09538388
  0.08535434 0.0743262  0.08385232 0.1116368 ]
 [0.09989547 0.08672333 0.16369636 0.14061093 0.06822471 0.07501411
  0.07739473 0.07819007 0.1387355  0.07151485]
 [0.03997786 0.08801058 0.07684112 0.14124104 0.07483738 0.15095049
  0.09787844 0.08921952 0.14267595 0.09836762]
 [0.1080392  0.05939941 0.07360125 0.09630901 0.14093342 0.10410528
  0.07452909 0.08005916 0.14993487 0.11308927]
 [0.18469317 0.04911042 0.06130705 0.10252552 0.05768099 0.09060261
  0.07184988 0.12812358 0.11810786 0.13599893]
 [0.09068755 0.08354026 0.07148395 0.06145024 0.13513081 0.08047622
  0.08995018 0.07978218 0.1926173  0.11488128]
 [0.07524562 0.03922917 0.13008448 0.08768924 0.1

INFO:tensorflow:probabilities = [[0.03793999 0.10992787 0.07242511 0.07313928 0.11365455 0.110092
  0.09022465 0.12693553 0.1341388  0.13152222]
 [0.17430112 0.06010123 0.10644577 0.12274789 0.07594846 0.07660069
  0.08955896 0.05969732 0.13942365 0.095175  ]
 [0.0597072  0.06583573 0.05009891 0.09266385 0.10153178 0.07504735
  0.06502607 0.2649019  0.09050573 0.13468154]
 [0.43640175 0.01191893 0.16760336 0.09192695 0.05236078 0.0399535
  0.04583689 0.04254324 0.04709801 0.06435659]
 [0.0526906  0.04384443 0.0389982  0.10863574 0.1552286  0.08256707
  0.09475957 0.18539917 0.05873482 0.17914188]
 [0.04235876 0.03726852 0.03678854 0.07510402 0.12899078 0.05304534
  0.07838093 0.2816761  0.05996308 0.20642392]
 [0.04262653 0.03412459 0.06909946 0.06222381 0.2495791  0.05629553
  0.11493772 0.13269144 0.08809289 0.15032892]
 [0.07480469 0.01538606 0.08919922 0.03605576 0.30537796 0.05327194
  0.15425645 0.09256636 0.04271653 0.13636501]
 [0.02148207 0.19700173 0.07768029 0.08209113 0.126

INFO:tensorflow:probabilities = [[0.0582175  0.02706824 0.38433316 0.14365117 0.06338346 0.0409212
  0.12084152 0.04622954 0.06744282 0.04791144]
 [0.05503481 0.13813299 0.04730142 0.12124138 0.05244857 0.05683588
  0.18228322 0.06876847 0.21621642 0.06173689]
 [0.1565378  0.05121548 0.07968688 0.05601053 0.14078078 0.09865555
  0.11574847 0.09369452 0.09909265 0.10857739]
 [0.10848588 0.0738176  0.07497482 0.1592114  0.13648392 0.10872371
  0.09177621 0.05281764 0.14387235 0.04983646]
 [0.06552072 0.05944698 0.1755545  0.24368525 0.06293349 0.08931804
  0.12131427 0.02355229 0.1297087  0.02896576]
 [0.25900224 0.02714361 0.35498542 0.05895942 0.04169347 0.04876008
  0.14885403 0.01584936 0.03665574 0.00809659]
 [0.01608507 0.06021731 0.08450781 0.26079923 0.05409144 0.08437362
  0.03708359 0.07298634 0.2087183  0.12113729]
 [0.0512103  0.09599169 0.08857987 0.11611443 0.05355516 0.06668907
  0.03187686 0.22003224 0.14075425 0.13519612]
 [0.03636406 0.36704978 0.06546469 0.09816107 0.0

INFO:tensorflow:probabilities = [[0.7540521  0.00142273 0.02464494 0.04381565 0.0137389  0.09581596
  0.01371892 0.01973214 0.02405071 0.00900799]
 [0.02154074 0.01801571 0.05040414 0.0186323  0.05937253 0.0526792
  0.6536817  0.02959763 0.03864145 0.05743473]
 [0.04179843 0.05873396 0.07894459 0.26784858 0.06460096 0.05087653
  0.05983906 0.10643385 0.21695317 0.05397085]
 [0.06892425 0.02190246 0.04090312 0.09711621 0.09569599 0.12004656
  0.13817382 0.18684633 0.04060053 0.18979077]
 [0.05271536 0.04662068 0.28285852 0.08724005 0.10639008 0.08433586
  0.08435348 0.04786694 0.11803667 0.08958238]
 [0.0376727  0.07176924 0.02870849 0.10729659 0.08496118 0.05344335
  0.03686577 0.3199177  0.10291583 0.15644915]
 [0.0149083  0.05293151 0.03722524 0.05409661 0.12760039 0.06155685
  0.08637761 0.1270326  0.08646297 0.3518079 ]
 [0.02876693 0.017387   0.0893736  0.01134823 0.10760667 0.02120902
  0.5161136  0.04044987 0.07252849 0.09521663]
 [0.5606948  0.01643709 0.02652406 0.037094   0.0

...

INFO:tensorflow:probabilities = [[0.02572755 0.12314212 0.13203472 0.05436583 0.12962492 0.04090913
  0.15294585 0.03205368 0.10988701 0.19930924]
 [0.00443588 0.18427329 0.55792874 0.02497076 0.00922467 0.01388254
  0.1185242  0.00757284 0.07339679 0.00579027]
 [0.00287095 0.68010914 0.06326065 0.03208319 0.0066925  0.04369375
  0.02125063 0.03133988 0.10130694 0.01739227]
 [0.09832905 0.00811389 0.11717658 0.54000854 0.01106947 0.11310825
  0.04086027 0.01078513 0.05116038 0.0093884 ]
 [0.9721534  0.00000328 0.00872007 0.00519145 0.00065885 0.00573484
  0.00266409 0.00188347 0.00047401 0.00251656]
 [0.02496027 0.01913432 0.06168361 0.5813162  0.0025025  0.06083529
  0.00329833 0.00556271 0.23293892 0.00776794]
 [0.08741849 0.01169575 0.26648715 0.31294852 0.0354824  0.12179016
  0.03855509 0.02059887 0.07742267 0.02760093]
 [0.95916045 0.00017394 0.00734933 0.01416563 0.0017826  0.01225919
  0.00177385 0.00055319 0.00219323 0.00058865]
 [0.00765328 0.16090558 0.43232393 0.1439811  0.

INFO:tensorflow:probabilities = [[0.05730325 0.04925098 0.11437514 0.19609158 0.02965801 0.21641295
  0.12825957 0.02160824 0.14214776 0.04489255]
 [0.10217999 0.00416215 0.34045127 0.00520749 0.06767644 0.02087939
  0.41452983 0.00272552 0.02710896 0.01507894]
 [0.00613869 0.00082822 0.00468107 0.01855794 0.02934235 0.69781876
  0.00174285 0.02190725 0.2129107  0.00607215]
 [0.00264362 0.00450333 0.00351918 0.02550545 0.08653425 0.05138652
  0.00661486 0.19022769 0.19750369 0.43156147]
 [0.8921586  0.00011508 0.02799873 0.03675343 0.00058741 0.03256937
  0.00431149 0.00102956 0.00330822 0.00116806]
 [0.01162127 0.00504499 0.04053837 0.00200314 0.02427387 0.01515317
  0.88466924 0.00035845 0.00519217 0.01114529]
 [0.30659848 0.00123514 0.27887926 0.0615297  0.03646476 0.09800114
  0.06641524 0.01880979 0.07685272 0.05521385]
 [0.00298484 0.01709782 0.02877185 0.29305115 0.06969757 0.05581453
  0.03485513 0.12254858 0.14137763 0.23380096]
 [0.00610654 0.525621   0.02014624 0.05177863 0.

INFO:tensorflow:probabilities = [[0.05194074 0.00117891 0.37304163 0.02406933 0.17390288 0.03367155
  0.08724976 0.0352358  0.00942796 0.21028137]
 [0.00984447 0.02567767 0.03536616 0.00594278 0.5594744  0.02748912
  0.22785597 0.0213778  0.03174668 0.055225  ]
 [0.32455665 0.00059445 0.41417983 0.01012987 0.01423567 0.0442109
  0.06531916 0.09666055 0.0093001  0.02081288]
 [0.00451622 0.00387391 0.03696596 0.00418526 0.01420342 0.00957207
  0.9151003  0.00145324 0.00650475 0.00362483]
 [0.00259492 0.0016922  0.03860433 0.00184364 0.04408439 0.00755992
  0.8912166  0.00179433 0.00584026 0.00476942]
 [0.00162375 0.47438774 0.03602128 0.05441518 0.03162189 0.0570182
  0.08560605 0.0990006  0.11288221 0.04742309]
 [0.03722509 0.00175923 0.00663288 0.04366518 0.07273629 0.15875965
  0.02285775 0.3016546  0.03757605 0.31713328]
 [0.10820311 0.0041946  0.02466836 0.00229408 0.4273045  0.01209932
  0.01099078 0.04034473 0.0946821  0.2752184 ]
 [0.00401303 0.00211086 0.02080735 0.76050186 0.00

INFO:tensorflow:probabilities = [[0.001343   0.91132885 0.00760535 0.01564756 0.00471886 0.00452483
  0.00665715 0.00856866 0.02968176 0.00992398]
 [0.00206079 0.5833722  0.01247412 0.09125738 0.02204354 0.03616452
  0.05894282 0.05388791 0.07625794 0.06353882]
 [0.00295236 0.00074805 0.95846957 0.01193108 0.00146632 0.002684
  0.01810136 0.00015746 0.00246768 0.00102216]
 [0.0409516  0.00172796 0.08937061 0.00344818 0.02500986 0.01848166
  0.8091053  0.00046862 0.00573113 0.00570497]
 [0.0002205  0.00394545 0.0015559  0.00450699 0.16211858 0.00603995
  0.02259555 0.04705752 0.01839286 0.7335667 ]
 [0.00080352 0.00002503 0.001033   0.00005173 0.9797158  0.00118968
  0.00225808 0.00521984 0.00184745 0.00785593]
 [0.23613992 0.00251752 0.16262826 0.11083    0.00897957 0.1681427
  0.12059159 0.04973173 0.10077822 0.03966057]
 [0.00716958 0.19750105 0.15832977 0.02085445 0.03915593 0.04330392
  0.4158487  0.00770011 0.08641645 0.02372   ]
 [0.00091171 0.82105356 0.00700246 0.02360751 0.007

INFO:tensorflow:probabilities = [[0.21202503 0.00412877 0.5418175  0.01375604 0.00093037 0.05147918
  0.08490774 0.00105032 0.08816947 0.00173557]
 [0.01439042 0.00603017 0.07278477 0.01423044 0.01829085 0.0285482
  0.80182266 0.00018572 0.02807381 0.01564289]
 [0.00370766 0.00016817 0.04375155 0.00907756 0.10702413 0.00835433
  0.12088707 0.02308539 0.00257166 0.68137246]
 [0.00162522 0.6051397  0.06451831 0.06260133 0.01236199 0.01804952
  0.03884679 0.08055715 0.07726518 0.03903485]
 [0.00265818 0.01739408 0.1247314  0.00634017 0.07238435 0.0153219
  0.7004799  0.00021659 0.04321842 0.01725488]
 [0.00582702 0.00063309 0.0039162  0.8981956  0.00006017 0.03312017
  0.0002245  0.01620295 0.04029293 0.00152736]
 [0.00050545 0.61684877 0.0091104  0.04712    0.00989484 0.02176332
  0.02377158 0.02899515 0.04354211 0.19844833]
 [0.04348649 0.00265569 0.02296679 0.6545291  0.00030401 0.03886273
  0.0109279  0.08301082 0.14046429 0.00279214]
 [0.0392927  0.00340572 0.0040925  0.22723155 0.00

In [44]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-16-12:28:15
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-2200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-16-12:28:16
INFO:tensorflow:Saving dict for global step 2200: accuracy = 0.8634, global_step = 2200, loss = 0.5684469
{'accuracy': 0.8634, 'loss': 0.5684469, 'global_step': 2200}
